# Understanding Steady-State Block

The model will have several parts. To make it easily digestable, we provide here a focused overview of the first component of the model: the steady state block.
$~$

The top-level function in the steady-state block is `call_find_steadystate`. This function will take as an argument a `struct` of parameters, which is propagated through the many sub-functions to generate the steady state of the model.
$~$

In trying to understand the steady-state block and its code, this jupyter notebook will propose different parameterizations and see how it affects the steady-state of the model. The notebook will also provide some intuition on the model mechanisms and functions at play. 
$~$

Below, I provide a roadmap on what to expect in this notebook.
$~$

First part of the notebook includes:
- Preliminairies (defining all relevant paths, generating structs, importing packages)
- Defining some functions we will need for comparing steady-states
- Finding the steady state 
- Plotting distributions of liquid assets, illiquid assets, and income for the baseline parameterization
$~$

Second part of the notebook will be on changing one parameter and seeing the effect of this change on the steady-state.
$~$

In the first block (a), we make a change to the household's preferences, answering questions such as: 
- What if there is a change to the risk aversion parameter?
- ... to the discount factor?
$~$

Second block (b) will be on the income process, to answer questions such as:
- What if income were less persistent?
- What if the standard deviation of the income shocks increase?
- What if the tax system becomes more progressive?
$~$

Last block (c) will be on the technology, to answer questions such as:
- What would greater depreciation imply for households?
- What if there is a change wage markups?
$~$

In the end, we will compare the resulting distributions side by side and provide some model intuition on what we see.

# 1.1 Preliminairies

In [ ]:
using PrettyTables, Printf;

## ------------------------------------------------------------------------------------------
## Header: set up paths, pre-process user inputs, load module
## ------------------------------------------------------------------------------------------

root_dir = replace(Base.current_project(), "Project.toml" => "");
cd(root_dir);

# set up paths for the project
paths = Dict(
    "root" => root_dir,
    "src" => joinpath(root_dir, "src"),
    "bld" => joinpath(root_dir, "bld"),
    "src_example" => @__DIR__,
    "bld_example" => replace(@__DIR__, "examples" => "bld") * "_noestim",
);

# create bld directory for the current example
mkpath(paths["bld_example"]);

# pre-process user inputs for model setup
include(paths["src"] * "/Preprocessor/PreprocessInputs.jl");
include(paths["src"] * "/BASEforHANK.jl");
using .BASEforHANK;

# set BLAS threads to the number of Julia threads, prevents grabbing all
BASEforHANK.LinearAlgebra.BLAS.set_num_threads(Threads.nthreads());

## ------------------------------------------------------------------------------------------
## Initialize: set up model parameters, priors, and estimation settings
## ------------------------------------------------------------------------------------------

# model parameters and priors
# model parameters
m_par = ModelParameters();
e_set = BASEforHANK.e_set;

# set some paths
@set! e_set.save_mode_file = paths["bld_example"] * "/HANK_mode.jld2";
@set! e_set.mode_start_file = paths["src_example"] * "/Data/par_final_dict.txt";

# fix seed for random number generation
BASEforHANK.Random.seed!(e_set.seed);

# 1.2 Functions we will need

In [ ]:
using LaTeXStrings

function optimal_layout(num_subplots::Int)
    rows = floor(Int, sqrt(num_subplots))
    cols = ceil(Int, num_subplots / rows)
    return rows, cols
end


function plot_steady_state_comparisons(ss_dict, income_grid, liquid_grid, illiquid_grid)
    n_structs = length(ss_dict)
    n_rows, n_cols = optimal_layout(n_structs)

    p1 = plot(layout=(n_rows, n_cols), size=(1400, 600), margin=10 * Plots.mm)
    p2 = plot(layout=(n_rows, n_cols), size=(1400, 600), margin=10 * Plots.mm)
    p3 = plot(layout=(n_rows, n_cols), size=(1400, 600), margin=10 * Plots.mm)
    p4 = plot(layout=(n_rows, n_cols))

    i = 1
    for (name, ss_full) in ss_dict

        # Compute distributions
        income_distribution = sum(ss_full.distrSS, dims=(1, 2))[:]
        liquid_distribution = sum(ss_full.distrSS, dims=(2, 3))[:]
        illiquid_distribution = sum(ss_full.distrSS, dims=(1, 3))[:]
        joint_liq_illiq_distribution = sum(ss_full.distrSS, dims=3)[:, :]

        # Create individual subplots for each steady state
        plot!(p1,
            income_grid,
            income_distribution,
            xformatter=:latex,
            yformatter=:latex,
            seriestype=:bar,
            label=L"\textrm{%$(name)}",
            xlabel=L"\textrm{Income}",
            ylabel=L"\textrm{Probability}",
            legend=:topright,
            subplot=i,
        )

        plot!(p2,
            liquid_grid,
            liquid_distribution,
            seriestype=:bar,
            label=L"\textrm{%$(name)}",
            xlabel=L"\textrm{Liquid \,\, Assets}",
            ylabel=L"\textrm{Probability}",
            legend=:topright,
            subplot=i,
            xformatter=:latex,
            yformatter=:latex
        )

        plot!(p3,
            illiquid_grid,
            illiquid_distribution,
            seriestype=:bar,
            label=L"\textrm{%$(name)}",
            xlabel=L"\textrm{Illiquid \,\, Assets}",
            ylabel=L"\textrm{Probability}",
            legend=:topright,
            subplot=i,
            xformatter=:latex,
            yformatter=:latex
        )

        surface!(p4, liquid_grid, illiquid_grid, joint_liq_illiq_distribution,
            xlabel=L"\textrm{Liquid \,\, Assets}",
            ylabel=L"\textrm{Illiquid \,\, Assets}",
            zlabel=L"\textrm{Probability \,\, Density}",
            xformatter=:latex,
            yformatter=:latex,
            zformatter=:latex,
            legend=false,
            camera=(75, 20),  # Viewing angle
            size=(1400, 600),
            colorbar=false,
            color=:viridis,
            subplot=i,
        )

        i += 1
    end

    return p1, p2, p3, p4
end

# 1.3 Finding the steady state

In [ ]:
@printf "Calculating the steady state\n";
ss_dict = Dict()
ss_dict["baseline"] = call_find_steadystate(m_par);

# 1.4 Plotting Distributions

The idiosyncratic states are:
1. `b`: liquid assets,
2. `k`: illiquid assets,
3. `h`: labor income.

$~~~~$

Computing the (marginal) distribution of, for example, income with then amount to integrating out the other states, liquid and illiquid assets, and plotting that along the income grid.

$~$
Below are the distributions from the baseline case.

In [ ]:
using Plots

# Defining grids
income_grid = log.(ss_dict["baseline"].n_par.grid_h .+ 1)
liquid_grid = sign.(ss_dict["baseline"].n_par.grid_b) .* log.(abs.(ss_dict["baseline"].n_par.grid_b) .+ 1)
illiquid_grid = sign.(ss_dict["baseline"].n_par.grid_k) .* log.(abs.(ss_dict["baseline"].n_par.grid_k) .+ 1)

p1, p2, p3, p4 = plot_steady_state_comparisons(ss_dict, income_grid, liquid_grid, illiquid_grid)

display(plot(p2))
display(plot(p3))
display(plot(p4))

# 2 Comparative Statics

We will go through different model blocks, each presented as its own subsection, and change some parameter values within each block. 

To facilitate comparisons across different parameterizations, we are going to store the different steady states and plot. 

To make a parameter change, we just need to do the following:

`@set! m_par.some_parameter = new_value`

### 2.1 What if household preferences change? 

In the model, there are four parameters that govern household preferences:
- risk aversion: ξ = 4.0
- inverse Frisch elasticity: γ = 2.0
- discount factor: β = 0.98255 
- adjustment probability: λ = 0.065 
$~~~~$

Below, we are going to see what happens in the model if you increase the **risk aversion parameter** by 50\% and then decrease by 50\% from baseline (from `4.0` to `6.0` to `2.0`).

In [ ]:
# Running the different scenarios
@set! m_par.ξ = 6.0
ss_dict["50p Increase in RA"] = call_find_steadystate(m_par);

@set! m_par.ξ = 2.0
ss_dict["50p Decrease in RA"] = call_find_steadystate(m_par);

### 2.2 What if income were less persistent?

In the model, there are four main parameters that govern the income process of households:
- autocorrelation income shock: ρ_h = 0.98
- std of income shocks (steady state): σ_h = 0.12
- probability to return worker: ι = 1 / 16
- probability to become entrepreneur: ζ = 1 / 4500
$~~~~$

Below, we are going to see what happens in the model if you decrease the **persistence parameter** by 25\% and then decrease by 50\% from baseline (from `0.98` to `0.75` to `0.50`).


In [ ]:
# Running the different scenarios
@set! m_par.ρ_h = 0.75
ss_dict["25p Decrease in Rho"] = call_find_steadystate(m_par);

@set! m_par.ρ_h = 0.5
ss_dict["50p Decrease in Rho"] = call_find_steadystate(m_par);

### 2.3  What if we change markups?
There are several parameters that govern the technology of the economy:
- capital share: α = 0.318 
- depreciation rate: δ_0 = (0.07 + 0.016) / 4 
- price markup: μ = 1.1 
- price adjustment costs (in terms of Calvo probs.): κ = 0.1456082664986374 
- wage markup: μw = 1.1 
- wage adjustment costs (in terms of Calvo probs.): κw = 0.23931075416274708 
$~~~~$

Below, we are going to see what happens in the model if you increase the **wage markup** by 50\% and then decrease by 50\% from baseline (from `1.1` to `1.65` to `0.55`).


In [ ]:
# Running the different scenarios
@set! m_par.μw = 1.65
ss_dict["50p Increase in Markups"] = call_find_steadystate(m_par);

@set! m_par.μw = 0.55
ss_dict["50p Decrease in Markups"] = call_find_steadystate(m_par);

In [ ]:
# Comparing all steady states now
p1, p2, p3, p4 = plot_steady_state_comparisons(sort(ss_dict), income_grid, liquid_grid, illiquid_grid)

display(plot(p2))
display(plot(p3))
display(plot(p4))

# 3 Model Intuition

### What if household preferences change?
$~$
#### **Model intution**
The risk aversion parameter ($\xi$) determines how households value consumption variability, directly influencing their precautionary savings behavior. Higher $\xi$ implies greater aversion to risk, making households prioritize smoothing consumption over time and across uncertain income states. This induces households to increase their liquid asset holdings, which are more accessible during shocks, but also increase their illiquid asset holdings. The increase in asset position can be observed above, with liquid and illiquid assets (on average) being greater relative to baseline. In economic terms, higher risk aversion enhances precautionary motives, shifting liquid and illiquid asset distributions to the right. These shifts reflect households' preference for any kind of savings as a buffer against uncertainty. **As an exercise**, one could see whether responses are linear in the parameters i.e., would studying a $25\%$ increase risk aversion elicit the same responses in liquid and illiquid assets as a $50\%$ increase, just divided by 2? Will households accumulate liquid assets and decumulate illiquid assets? Is this co-movement in assets robust across all parametizations?

$~~$

#### **Functions involved**
The first key function influenced by $\xi$ is the utility function of the households $u(c) = \frac{c^{1-\xi}}{1-\xi} $, which reflects the standard constant relative risk aversion (CRRA) specification. Here, $\xi$ represents the coefficient of relative risk aversion.

The second key function will be used when solving the household problem. We use the Endogenous Grid Method (EGM) to solve the household problem, which leverages the first-order conditions of the problem and Euler equations to determine optimal choices. Here, $\xi$ affects the marginal utility of consumption, which directly depends on ξ, affecting the household's intertemporal trade-offs between current and future consumption. Specifically, `EGM_policyupdate` determines optimal consumption and savings policies, influenced by the marginal utility of consumption $u'(c) = c^{-\xi}$. Changes in these policies will ultimately affect the steady-state asset distributions (`distrSS`).


### What if income were less persistent?
$~$
#### **Model Intuition**

The autocorrelation parameter of the income process ($\rho_h $) determines the persistence of idiosyncratic income changes over time. A higher  $\rho_h$ implies that income is more persistent, meaning a positive or negative shock to income is likely to have long-lasting effects. This alters households’ expectations about future income, reducing the need for precautionary savings when $\rho_h$ increases, as households expect a more predictable income trajectory. Conversely, lower $\rho_h$ (less persistence) increases uncertainty, but what does the degree of lesser persistence mean for wealth accumulation? A $25\%$ decrease in $\rho_h$ shrunk both the liquid and illiquid assets distribution to around zero, implying households are beginning to pay the cost of greater uncertainty. A $50\%$ decrease, however, introduces greater income uncertainty, which implies even greater shrinkage in assets. One could mess around with the standard deviation of the income shocks to see its transmission on asset holdings. 

$~~$

#### **Functions Involved**

The parameter $\rho_h$ affects the income process, specifically the transition matrix for income states (`n_par.Π`), under the functions `Fsys` and `VFI`. This matrix is calculated using functions like `ExTransition`, which calculates a transition probability matrix for a discretized stochastic income process, similar to the widely used Tauchen method for approximating an AR(1). Of course, the income process of households has implications for their consumption and savings decisions and thus, all functions related to solving the household problem (as mentioned above) apply.


### What if there is a change in wage markups?
$~$
#### **Model Intuition**
The wage markup ($\mu_w$) determines the degree of market power that workers have in setting wages above the competitive level. Increasing $\mu_w$ by 50% (to 1.65) implies workers can extract higher wages relative to their productivity, increasing labor costs for firms. Firms will most likely respond by reducing employment (depending on the persistence of wage markup). This reduces household disposable income for saving, leading to liquid and illiquid asset distributions around zero. Conversely, decreasing $\mu_w$ by 50% (to 0.55) reduces wage-setting power, making labor cheaper and encouraging higher employment. This increases household income and saving capacity, shifting the asset distributions outward, but only slightly. The baseline appears to grant greater capacity for household saving.

$~~$

#### **Functions Involved**

The parameter $\mu_w$ appears in wage-setting equations, for example: the Wage Phillips curve
$$
\begin{align}
    \log \pi_{w,t} - \pi_w = \kappa_w \left(mc_{w,t} - \frac{1}{\mu_w} \right) + \beta \left(( \log \pi_{w,t+1} - \pi_w) \frac{N_{t+1} w_{t+1}}{N_t w_t} \right)
\end{align}
$$ 
$\mu_w$ also affects union profits (`profits_U`), which are calculated in functions like `incomes_net` and `wage`. Changes in $\mu_w $ directly impact the steady-state income distribution (`distrSS`) and marginal utility of consumption via the disposable income households receive. These adjustments flow through the Endogenous Grid Method (`EGM_policyupdate`) as already aforementioned, altering optimal consumption and savings decisions. 
